In [77]:
import pandas as pd

In [78]:
base_estoque = pd.read_excel("bases/bases_limpas/base_estoque_limpo.xlsx")
base_notas = pd.read_excel("bases/bases_limpas/base_notas_limpa.xlsx")

Exception ignored in: <function ZipFile.__del__ at 0x000002589FA351C0>
Traceback (most recent call last):
  File "C:\Users\Paulo Neto\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 1975, in __del__
    self.close()
  File "C:\Users\Paulo Neto\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 1992, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [79]:
base_notas.columns.tolist()

['Numero nota fiscal',
 'Data da venda',
 'Código produto',
 'Descrição do produto',
 'Quantidade do produto',
 'Valor unitário',
 'Preço de custo',
 'Valor total produto',
 'Valor da nota']

In [80]:
base_estoque.columns.tolist()

['Código produto',
 'Produto',
 'Código da categoria',
 'Categoria',
 'Código da Marca',
 'Marca',
 'Preço de custo',
 'Quantidade estoque']

In [81]:
df = pd.merge(
    base_notas,
    base_estoque[["Código produto", "Código da categoria", "Categoria", "Código da Marca", "Marca", "Quantidade estoque"]],
    on='Código produto',
    how='inner' 
)

In [82]:
df.columns.tolist()

['Numero nota fiscal',
 'Data da venda',
 'Código produto',
 'Descrição do produto',
 'Quantidade do produto',
 'Valor unitário',
 'Preço de custo',
 'Valor total produto',
 'Valor da nota',
 'Código da categoria',
 'Categoria',
 'Código da Marca',
 'Marca',
 'Quantidade estoque']

In [83]:
df.nunique()

Numero nota fiscal       24353
Data da venda              135
Código produto            5634
Descrição do produto      5634
Quantidade do produto      420
Valor unitário            7885
Preço de custo            2833
Valor total produto      19702
Valor da nota            20794
Código da categoria        382
Categoria                  382
Código da Marca            305
Marca                      305
Quantidade estoque         995
dtype: int64

In [84]:
df.head(5)

,Numero nota fiscal,Data da venda,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Preço de custo,Valor total produto,Valor da nota,Código da categoria,Categoria,Código da Marca,Marca,Quantidade estoque
0,566082,17/06/2025,33932,ENV 229X324 OU C/100 FILIPERSON,2.0,33.00,24.41,66.00,4217.74,47,ENVELOPES ( ENV ),8389,FILIPERSON NACIONAL REFL IND COMÉRCIO DE PAP E...,2076
1,566082,17/06/2025,29367,GUARDANAPO SNOB FS 33X30 50F SANTHER,5.0,2.90,2.23,14.50,4217.74,27,GUARDANAPOS ( GDP ),8340,SANTHER FABRICA DE PAPEL SANTA THEREZINHA S/A,3475
2,566082,17/06/2025,241,PASTA AZ OF LE TIGRADA FRAMA 356-11952,40.0,9.45,7.61,378.00,4217.74,38,PASTAS ( PST ),185,FRAMA INDUSTRIA GRAFICA LTDA,2104
3,566082,17/06/2025,32608,CALCULADORA MESA 12 DIG MASTERPRINT 1086,6.0,20.87,17.05,125.22,4217.74,147,CALCULADORAS ( CLC ),4281,GRUPO MASTER COMERCIO IMPORTACAO E EXPORTACAO ...,61
4,566082,17/06/2025,25495,MARCA TEXTO AM C/12 MASTERPRINT MP612,2.0,11.30,8.72,22.60,4217.74,389,MARCA TEXTO ( MRT),4281,GRUPO MASTER COMERCIO IMPORTACAO E EXPORTACAO ...,293


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178107 entries, 0 to 178106
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Numero nota fiscal     178107 non-null  int64  
 1   Data da venda          178107 non-null  object 
 2   Código produto         178107 non-null  int64  
 3   Descrição do produto   178107 non-null  object 
 4   Quantidade do produto  178107 non-null  float64
 5   Valor unitário         178107 non-null  float64
 6   Preço de custo         177320 non-null  float64
 7   Valor total produto    177039 non-null  float64
 8   Valor da nota          178107 non-null  float64
 9   Código da categoria    178107 non-null  int64  
 10  Categoria              178107 non-null  object 
 11  Código da Marca        178107 non-null  int64  
 12  Marca                  178107 non-null  object 
 13  Quantidade estoque     178107 non-null  int64  
dtypes: float64(5), int64(5), object(4)
m

In [86]:
df["Data da venda"] = pd.to_datetime(df["Data da venda"], format='%d/%m/%Y')
df["Quantidade do produto"] = df["Quantidade do produto"].astype(int)
df["Código da categoria"] = df["Código da categoria"].astype(int)
df["Código da Marca"] = df["Código da Marca"].astype(int)
df["Quantidade estoque"] = df["Quantidade estoque"].astype(int)


In [87]:
df.dtypes

Numero nota fiscal                int64
Data da venda            datetime64[ns]
Código produto                    int64
Descrição do produto             object
Quantidade do produto             int64
Valor unitário                  float64
Preço de custo                  float64
Valor total produto             float64
Valor da nota                   float64
Código da categoria               int64
Categoria                        object
Código da Marca                   int64
Marca                            object
Quantidade estoque                int64
dtype: object

In [88]:
df.isnull().sum()

Numero nota fiscal          0
Data da venda               0
Código produto              0
Descrição do produto        0
Quantidade do produto       0
Valor unitário              0
Preço de custo            787
Valor total produto      1068
Valor da nota               0
Código da categoria         0
Categoria                   0
Código da Marca             0
Marca                       0
Quantidade estoque          0
dtype: int64

In [89]:
# filtrar linhas onde o preço de custo é nulo
itens_sem_preco_custo = df[df["Preço de custo"].isna()]
# extrair os números das notas com itens problemáticos
notas_afetadas = itens_sem_preco_custo["Numero nota fiscal"].unique()
# contar quantas notas são impactadas
total_notas_afetadas = len(notas_afetadas)
total_notas_unicas = df["Numero nota fiscal"].nunique()
percentual_afetado = (total_notas_afetadas / total_notas_unicas) * 100

# Resultados
print(f"Total de notas fiscais afetadas: {total_notas_afetadas}")
print(f"Total de notas únicas no dataset: {total_notas_unicas}")
print(f"Percentual de notas afetadas: {percentual_afetado:.2f}%")

Total de notas fiscais afetadas: 68
Total de notas únicas no dataset: 24353
Percentual de notas afetadas: 0.28%


In [90]:
# 1. Lista das notas fiscais que têm pelo menos um item com "Preço de custo" nulo
notas_com_problema = df[df["Preço de custo"].isna()]["Numero nota fiscal"].unique()

# 2. Criar um novo DataFrame excluindo essas notas
df_completo_limpo = df[~df["Numero nota fiscal"].isin(notas_com_problema)]

# 3. Verificar se ainda há valores nulos em "Preço de custo"
print("Valores nulos restantes:", df_completo_limpo["Preço de custo"].isna().sum())

# 4. Comparar tamanhos antes e depois
print(f"Total de notas antes: {df['Numero nota fiscal'].nunique()}")
print(f"Total de notas depois: {df_completo_limpo['Numero nota fiscal'].nunique()}")

Valores nulos restantes: 0
Total de notas antes: 24353
Total de notas depois: 24285


In [91]:

def verificar_base_mesclada(df):
    """
    Realiza uma análise completa da base mesclada para identificar possíveis problemas.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame mesclado a ser verificado
    
    Retorna:
        Relatório com possíveis problemas e sugestões de limpeza
    """
    print("🔍 INÍCIO DA ANÁLISE DA BASE MESCLADA 🔍\n")
    
    ## 1. Verificação básica da estrutura
    print("📊 INFORMAÇÕES BÁSICAS:")
    print(f"- Total de registros: {len(df):,}")
    print(f"- Total de notas fiscais únicas: {df['Numero nota fiscal'].nunique():,}")
    print(f"- Total de produtos únicos: {df['Código produto'].nunique():,}")
    print(f"- Período coberto: De {df['Data da venda'].min()} a {df['Data da venda'].max()}\n")
    
    ## 2. Verificação de valores nulos
    print("🕵️ VALORES NULOS POR COLUNA:")
    nulos = df.isnull().sum()
    display(nulos[nulos > 0].to_frame('Quantidade de Nulos'))
    
    ## 3. Verificação de valores zerados
    print("\n🔢 VALORES ZERADOS EM COLUNAS NUMÉRICAS:")
    colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    for col in colunas_numericas:
        zeros = (df[col] == 0).sum()
        if zeros > 0:
            print(f"- {col}: {zeros} zeros ({zeros/len(df):.1%})")
    
    ## 4. Verificação de valores negativos
    print("\n📉 VALORES NEGATIVOS EM COLUNAS NUMÉRICAS:")
    for col in colunas_numericas:
        negativos = (df[col] < 0).sum()
        if negativos > 0:
            print(f"- {col}: {negativos} negativos")
    
    ## 5. Verificação de consistência de valores
    print("\n🧮 VERIFICAÇÃO DE CONSISTÊNCIA:")
    # Verifica se Valor total produto = Quantidade * Valor unitário
    inconsistencias_valor = (df['Valor total produto'].round(2) != 
                            (df['Quantidade do produto'] * df['Valor unitário']).round(2)).sum()
    print(f"- Inconsistências no cálculo do valor total: {inconsistencias_valor} registros")
    
    # Verifica se Preço de custo é maior que Valor unitário (possível erro)
    custo_maior = (df['Preço de custo'] > df['Valor unitário']).sum()
    print(f"- Itens com preço de custo maior que valor de venda: {custo_maior} registros")
    
    ## 6. Verificação de relacionamentos
    print("\n🔗 VERIFICAÇÃO DE RELACIONAMENTOS:")
    # Produtos com múltiplas categorias
    produtos_mult_cat = df.groupby('Código produto')['Código da categoria'].nunique()
    print(f"- Produtos com múltiplas categorias: {sum(produtos_mult_cat > 1)}")
    
    # Produtos sem estoque
    sem_estoque = (df['Quantidade estoque'] == 0).sum()
    print(f"- Itens com estoque zerado: {sem_estoque} ({sem_estoque/len(df):.1%})")
    
    ## 7. Verificação de datas
    print("\n📅 VERIFICAÇÃO DE DATAS:")
    datas_futuras = (df['Data da venda'] > pd.Timestamp.now()).sum()
    print(f"- Vendas com data no futuro: {datas_futuras}")
    
    ## 8. Sugestões de limpeza
    print("\n💡 SUGESTÕES DE LIMPEZA:")
    problemas = []
    if nulos.sum() > 0:
        problemas.append("Tratar valores nulos (preencher ou excluir)")
    if inconsistencias_valor > 0:
        problemas.append("Corrigir cálculos de valor total")
    if custo_maior > 0:
        problemas.append("Investigar itens com preço de custo > valor de venda")
    if sem_estoque > 0:
        problemas.append("Analisar itens com estoque zerado")
    
    if problemas:
        for i, problema in enumerate(problemas, 1):
            print(f"{i}. {problema}")
    else:
        print("A base parece estar consistente. Nenhuma ação crítica necessária.")
    
    print("\n✅ ANÁLISE CONCLUÍDA")

# Uso: (assumindo que sua base mesclada se chama 'df_mesclado')
# verificar_base_mesclada(df_mesclado)

In [92]:
verificar_base_mesclada(df_completo_limpo)

🔍 INÍCIO DA ANÁLISE DA BASE MESCLADA 🔍

📊 INFORMAÇÕES BÁSICAS:
- Total de registros: 177,127
- Total de notas fiscais únicas: 24,285
- Total de produtos únicos: 5,602
- Período coberto: De 2025-01-02 00:00:00 a 2025-06-17 00:00:00

🕵️ VALORES NULOS POR COLUNA:


,Quantidade de Nulos
Valor total produto,97



🔢 VALORES ZERADOS EM COLUNAS NUMÉRICAS:
- Valor unitário: 85 zeros (0.0%)
- Preço de custo: 1 zeros (0.0%)
- Valor da nota: 97 zeros (0.1%)
- Quantidade estoque: 3620 zeros (2.0%)

📉 VALORES NEGATIVOS EM COLUNAS NUMÉRICAS:
- Quantidade do produto: 6 negativos

🧮 VERIFICAÇÃO DE CONSISTÊNCIA:
- Inconsistências no cálculo do valor total: 4658 registros
- Itens com preço de custo maior que valor de venda: 446 registros

🔗 VERIFICAÇÃO DE RELACIONAMENTOS:
- Produtos com múltiplas categorias: 0
- Itens com estoque zerado: 3620 (2.0%)

📅 VERIFICAÇÃO DE DATAS:
- Vendas com data no futuro: 0

💡 SUGESTÕES DE LIMPEZA:
1. Tratar valores nulos (preencher ou excluir)
2. Corrigir cálculos de valor total
3. Investigar itens com preço de custo > valor de venda
4. Analisar itens com estoque zerado

✅ ANÁLISE CONCLUÍDA


In [93]:
def verificar_valores_zerados(df):
    """
    Verifica valores zerados em colunas numéricas da base mesclada.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame a ser analisado
    
    Retorna:
        DataFrame com contagem de zeros por coluna numérica
    """
    # Seleciona apenas colunas numéricas
    colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    
    # Conta valores zerados em cada coluna numérica
    zeros_por_coluna = []
    for coluna in colunas_numericas:
        qtd_zeros = (df[coluna] == 0).sum()
        zeros_por_coluna.append({'Coluna': coluna, 'Valores Zerados': qtd_zeros})
    
    # Cria e retorna DataFrame com resultados
    resultado = pd.DataFrame(zeros_por_coluna)
    resultado['Percentual'] = (resultado['Valores Zerados'] / len(df) * 100).round(2)
    
    # Ordena por quantidade de zeros (decrescente)
    return resultado.sort_values('Valores Zerados', ascending=False)

# Como usar:
# resultado_zeros = verificar_valores_zerados(df_mesclado)
# display(resultado_zeros)

In [94]:
resultado_zeros = verificar_valores_zerados(df_completo_limpo)
display(resultado_zeros)

,Coluna,Valores Zerados,Percentual
9,Quantidade estoque,3620,2.04
6,Valor da nota,97,0.05
3,Valor unitário,85,0.05
4,Preço de custo,1,0.00
1,Código produto,0,0.00
0,Numero nota fiscal,0,0.00
5,Valor total produto,0,0.00
2,Quantidade do produto,0,0.00
7,Código da categoria,0,0.00
8,Código da Marca,0,0.00


In [95]:
# 1. Identificar as notas fiscais com Valor unitário zerado
notas_com_valor_zerado = df[df['Valor unitário'] == 0]['Numero nota fiscal'].unique()
total_notas_afetadas = len(notas_com_valor_zerado)

print(f"🔍 Notas fiscais com Valor unitário zerado: {total_notas_afetadas}")

# 2. Criar uma versão limpa do DataFrame excluindo essas notas
df_limpo = df[~df['Numero nota fiscal'].isin(notas_com_valor_zerado)].copy()

# 3. Recalcular o Valor total produto e Valor da nota
df_limpo['Valor total produto'] = df_limpo['Quantidade do produto'] * df_limpo['Valor unitário']
df_limpo['Valor da nota'] = df_limpo.groupby('Numero nota fiscal')['Valor total produto'].transform('sum')

# 4. Verificação final
print("\n✅ Resultado da limpeza:")
print(f"- Total de notas originais: {df['Numero nota fiscal'].nunique()}")
print(f"- Notas removidas: {total_notas_afetadas}")
print(f"- Notas restantes: {df_limpo['Numero nota fiscal'].nunique()}")
print(f"- Registros removidos: {len(df) - len(df_limpo)}")
print(f"- Novos valores calculados corretamente? {df_limpo['Valor total produto'].equals(df_limpo['Quantidade do produto'] * df_limpo['Valor unitário'])}")

# 5. (Opcional) Análise detalhada das notas removidas
if total_notas_afetadas > 0:
    print("\n📊 Detalhes das notas removidas:")
    notas_removidas = df[df['Numero nota fiscal'].isin(notas_com_valor_zerado)]
    display(notas_removidas.groupby(['Numero nota fiscal', 'Descrição do produto'])[['Quantidade do produto', 'Valor unitário']].sum())

🔍 Notas fiscais com Valor unitário zerado: 85

✅ Resultado da limpeza:
- Total de notas originais: 24353
- Notas removidas: 85
- Notas restantes: 24268
- Registros removidos: 670
- Novos valores calculados corretamente? True

📊 Detalhes das notas removidas:


Quantidade do produto  \
Numero nota fiscal Descrição do produto                                         
37033              ACUCAR CRISTAL 1KG SAMUKA                                0   
                   AGUA SANITARIA 5L TA LIMPEZA                             0   
                   AROMAT LAVANDA FLORAL 400ML LEV&UZE                      0   
                   CAFE TRADICIONAL VACUO 250G UNIAO                        0   
                   COLA INSTANTANEA 20G TEKBOND 725                         0   
...                                                                       ...   
564625             SACO 45X59 SORTIDO FEM C/25 CROMUS                       1   
                   SACO 45X59 SORTIDO MASC C/25 CROMUS                      1   
                   SACO 50X70 ROMANTICO C/25 CROMUS                         1   
565290             SUCATA DE PAPELAO                                        0   
565870             DISPLAY BALCAO SPIRO CIS                                 1   

                                                        Valor unitário  
Numero nota fiscal Descrição do produto                                 
37033              ACUCAR CRISTAL 1KG SAMUKA                       0.0  
                   AGUA SANITARIA 5L TA LIMPEZA                    0.0  
                   AROMAT LAVANDA FLORAL 400ML LEV&UZE             0.0  
                   CAFE TRADICIONAL VACUO 250G UNIAO               0.0  
                   COLA INSTANTANEA 20G TEKBOND 725                0.0  
...                                                                ...  
564625             SACO 45X59 SORTIDO FEM C/25 CROMUS              0.0  
                   SACO 45X59 SORTIDO MASC C/25 CROMUS             0.0  
                   SACO 50X70 ROMANTICO C/25 CROMUS                0.0  
565290             SUCATA DE PAPELAO                               0.0  
565870             DISPLAY BALCAO SPIRO CIS                        0.0  

[637 rows x 2 columns]

In [96]:
df_completo_limpo.isnull().sum()

Numero nota fiscal        0
Data da venda             0
Código produto            0
Descrição do produto      0
Quantidade do produto     0
Valor unitário            0
Preço de custo            0
Valor total produto      97
Valor da nota             0
Código da categoria       0
Categoria                 0
Código da Marca           0
Marca                     0
Quantidade estoque        0
dtype: int64

In [97]:
import pandas as pd

def analisar_valores_negativos(df):
    """
    Analisa valores negativos em todas as colunas numéricas da base mesclada.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame a ser analisado
    
    Retorna:
        DataFrame com análise detalhada dos valores negativos encontrados
    """
    print("🔍 ANÁLISE DE VALORES NEGATIVOS NA BASE MESCLADA")
    
    # 1. Identificar colunas numéricas
    colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    
    # 2. Verificar valores negativos em cada coluna
    resultados = []
    for coluna in colunas_numericas:
        negativos = df[df[coluna] < 0]
        qtd_negativos = len(negativos)
        
        if qtd_negativos > 0:
            # Coletar estatísticas
            qtd_notas_afetadas = negativos['Numero nota fiscal'].nunique()
            qtd_produtos_afetados = negativos['Código produto'].nunique()
            
            resultados.append({
                'Coluna': coluna,
                'Registros Negativos': qtd_negativos,
                'Notas Afetadas': qtd_notas_afetadas,
                'Produtos Afetados': qtd_produtos_afetados,
                'Valor Mínimo': df[coluna].min(),
                'Valor Máximo (neg)': df[df[coluna] < 0][coluna].max()
            })
    
    # 3. Exibir resultados
    if resultados:
        resultado_df = pd.DataFrame(resultados)
        resultado_df = resultado_df.sort_values('Registros Negativos', ascending=False)
        
        print("\n📊 RESUMO DE VALORES NEGATIVOS:")
        display(resultado_df)
        
        # 4. Detalhamento por coluna problemática
        for _, row in resultado_df.iterrows():
            coluna = row['Coluna']
            print(f"\n🔴 DETALHES PARA COLUNA '{coluna}':")
            
            # Mostrar exemplos dos registros problemáticos
            display(
                df[df[coluna] < 0][['Numero nota fiscal', 'Data da venda', 'Código produto', 
                                   'Descrição do produto', coluna]].head()
            )
            
            # Análise por produto
            produtos_neg = df[df[coluna] < 0].groupby(['Código produto', 'Descrição do produto'])[coluna].sum()
            print(f"\n📦 Produtos com maiores valores negativos em '{coluna}':")
            display(produtos_neg.sort_values().head(10))
            
            # Análise temporal
            if 'Data da venda' in df.columns:
                temp_neg = df[df[coluna] < 0].groupby(pd.to_datetime(df['Data da venda']).dt.to_period('M'))[coluna].count()
                print("\n📅 Distribuição temporal dos valores negativos:")
                display(temp_neg)
    else:
        print("\n✅ Nenhum valor negativo encontrado na base.")
    
    return resultado_df if resultados else None

# Como usar:
# resultados_negativos = analisar_valores_negativos(df_mesclado)

In [98]:
resultados_negativos = analisar_valores_negativos(df_completo_limpo)

🔍 ANÁLISE DE VALORES NEGATIVOS NA BASE MESCLADA

📊 RESUMO DE VALORES NEGATIVOS:


,Coluna,Registros Negativos,Notas Afetadas,Produtos Afetados,Valor Mínimo,Valor Máximo (neg)
0,Quantidade do produto,6,4,5,-160,-1



🔴 DETALHES PARA COLUNA 'Quantidade do produto':


,Numero nota fiscal,Data da venda,Código produto,Descrição do produto,Quantidade do produto
137760,544686,2025-01-21,24217,PALITO DENTE MADEIRA PCT/25X100 GINA,-160
139217,544445,2025-01-20,24217,PALITO DENTE MADEIRA PCT/25X100 GINA,-8
152195,542514,2025-01-08,24370,SACO LIXO PT 40L C/100 NOVA RAVA AHC920,-20
177832,496,2025-04-01,6620,BAND ISOPOR B3 BR C/400 COPOBRAS CRL003,-50
177833,496,2025-04-01,30643,TRAVESSEIRO SILK TOUCH 50X70 ALTENBURG,-1



📦 Produtos com maiores valores negativos em 'Quantidade do produto':


Código produto  Descrição do produto                   
24217           PALITO DENTE MADEIRA PCT/25X100 GINA      -168
35488           TNT TECIDO 50M 37G BR DECOFIX              -60
6620            BAND ISOPOR B3 BR C/400 COPOBRAS CRL003    -50
24370           SACO LIXO PT 40L C/100 NOVA RAVA AHC920    -20
30643           TRAVESSEIRO SILK TOUCH 50X70 ALTENBURG      -1
Name: Quantidade do produto, dtype: int64


📅 Distribuição temporal dos valores negativos:


Data da venda
2025-01    3
2025-04    3
Freq: M, Name: Quantidade do produto, dtype: int64

In [99]:
def excluir_notas_com_negativos(df, colunas_verificar=None):
    """
    Remove todas as notas fiscais que contêm valores negativos nas colunas especificadas.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados mesclados
        colunas_verificar (list): Lista de colunas para verificar valores negativos
                                 Se None, verifica todas as colunas numéricas
    
    Retorno:
        pd.DataFrame: DataFrame limpo sem as notas problemáticas
        pd.DataFrame: DataFrame com as notas removidas (para auditoria)
    """
    # 1. Definir colunas para verificação
    if colunas_verificar is None:
        colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
        colunas_verificar = [col for col in colunas_numericas if col not in ['Numero nota fiscal', 'Código produto']]
    
    # 2. Identificar registros com valores negativos
    mascara_negativos = pd.Series(False, index=df.index)
    for coluna in colunas_verificar:
        mascara_negativos |= (df[coluna] < 0)
    
    # 3. Identificar notas fiscais problemáticas
    notas_com_negativos = df.loc[mascara_negativos, 'Numero nota fiscal'].unique()
    print(f"🔍 Encontradas {len(notas_com_negativos)} notas fiscais com valores negativos")
    
    # 4. Criar DataFrames para resultado e auditoria
    df_limpo = df[~df['Numero nota fiscal'].isin(notas_com_negativos)].copy()
    df_removido = df[df['Numero nota fiscal'].isin(notas_com_negativos)].copy()
    
    # 5. Recalcular valores totais na base limpa
    df_limpo['Valor total produto'] = df_limpo['Quantidade do produto'] * df_limpo['Valor unitário']
    df_limpo['Valor da nota'] = df_limpo.groupby('Numero nota fiscal')['Valor total produto'].transform('sum')
    
    # 6. Relatório de impacto
    print("\n✅ Resultado da limpeza:")
    print(f"- Total original de notas: {df['Numero nota fiscal'].nunique()}")
    print(f"- Notas removidas: {len(notas_com_negativos)}")
    print(f"- Notas restantes: {df_limpo['Numero nota fiscal'].nunique()}")
    print(f"- Registros removidos: {len(df_removido)}")
    print(f"- Percentual removido: {len(df_removido)/len(df):.2%}")
    
    return df_limpo, df_removido

# Como usar:
# df_limpo, df_removido = excluir_notas_com_negativos(df_mesclado, colunas_verificar=['Quantidade do produto', 'Valor unitário', 'Preço de custo'])

# Opção para verificar todas colunas numéricas:
# df_limpo, df_removido = excluir_notas_com_negativos(df_mesclado)

In [100]:
df_limpo, df_removido = excluir_notas_com_negativos(df_completo_limpo)

🔍 Encontradas 4 notas fiscais com valores negativos

✅ Resultado da limpeza:
- Total original de notas: 24285
- Notas removidas: 4
- Notas restantes: 24281
- Registros removidos: 12
- Percentual removido: 0.01%


In [101]:
def verificar_custo_maior_que_venda(df):
    """
    Identifica itens onde o preço de custo é maior que o valor unitário de venda
    e analisa o impacto nas notas fiscais.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os dados mesclados
        
    Retorna:
        pd.DataFrame: Itens problemáticos com análise detalhada
    """
    print("🔍 ANALISANDO ITENS ONDE PREÇO DE CUSTO > VALOR UNITÁRIO")
    
    # 1. Criar filtro para itens problemáticos
    df_problema = df[df['Preço de custo'] > df['Valor unitário']].copy()
    
    # 2. Calcular prejuízo estimado
    df_problema['Prejuízo Unitário'] = df_problema['Preço de custo'] - df_problema['Valor unitário']
    df_problema['Prejuízo Total'] = df_problema['Prejuízo Unitário'] * df_problema['Quantidade do produto']
    
    # 3. Análise agregada
    resumo = df_problema.groupby(['Código produto', 'Descrição do produto']).agg(
        Qtde_Ocorrencias=('Código produto', 'count'),
        Notas_Afetadas=('Numero nota fiscal', 'nunique'),
        Custo_Medio=('Preço de custo', 'mean'),
        Venda_Media=('Valor unitário', 'mean'),
        Prejuízo_Total=('Prejuízo Total', 'sum')
    ).sort_values('Prejuízo_Total', ascending=False)
    
    # 4. Exibir resultados
    print(f"\n⚠️ {len(df_problema)} itens problemáticos encontrados")
    print(f"📊 Afetando {df_problema['Numero nota fiscal'].nunique()} notas fiscais")
    print(f"💸 Prejuízo total estimado: R${df_problema['Prejuízo Total'].sum():,.2f}\n")
    
    print("📌 RESUMO POR PRODUTO:")
    display(resumo.head(20))
    
    print("\n📝 DETALHES COMPLETOS DOS ITENS PROBLEMÁTICOS:")
    return df_problema.sort_values('Prejuízo Total', ascending=False)

# Como usar:
# itens_problematicos = verificar_custo_maior_que_venda(df_mesclado)
# itens_problematicos.to_excel('itens_custo_maior_que_venda.xlsx', index=False)

In [102]:
itens_problematicos = verificar_custo_maior_que_venda(df_limpo)

🔍 ANALISANDO ITENS ONDE PREÇO DE CUSTO > VALOR UNITÁRIO

⚠️ 440 itens problemáticos encontrados
📊 Afetando 272 notas fiscais
💸 Prejuízo total estimado: R$71,736.77

📌 RESUMO POR PRODUTO:


,,Qtde_Ocorrencias,Notas_Afetadas,Custo_Medio,Venda_Media,Prejuízo_Total
Código produto,Descrição do produto,,,,,
6620,BAND ISOPOR B3 BR C/400 COPOBRAS CRL003,1,1,61.910,8.830000,10616.00
2488,FOLHA ISOPOR 20MM C/14 ISOPLAST,2,2,54.550,24.240000,5756.30
1976,FOLHA ISOPOR 15MM C/18 ISOPLAST,2,2,52.710,23.450000,5557.60
36994,DISPLAY BALCAO SPIRO CIS,24,24,171.640,0.000000,4119.36
29340,SACO LIXO 100L PT 0.6 C/100 SORDI,2,2,42.140,16.390000,3862.50
2265,FOLHA ISOPOR 10MM C/26 ISOPLAST,2,2,50.585,22.510000,3312.50
29341,SACO LIXO 200L PT 0.6 C/100 SORDI,1,1,59.090,22.990000,2527.00
6619,BAND ISOPOR B2 BR C/400 COPOBRAS CRL002,1,1,35.760,28.300000,2051.50
26179,SACO LIXO 90X110X0.12 PT 200L MG,3,3,8.350,7.800000,1925.00



📝 DETALHES COMPLETOS DOS ITENS PROBLEMÁTICOS:


In [103]:
def excluir_notas_com_custo_maior_que_venda(df):
    """
    Remove todas as notas fiscais que contêm itens onde:
    Preço de custo > Valor unitário
    
    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados mesclados
        
    Retorna:
        pd.DataFrame: Base limpa sem as notas problemáticas
        pd.DataFrame: Notas removidas (para auditoria)
    """
    # 1. Identificar itens problemáticos
    mascara_problema = df['Preço de custo'] > df['Valor unitário']
    itens_problematicos = df[mascara_problema]
    
    # 2. Pegar números das notas fiscais problemáticas
    notas_problematicas = itens_problematicos['Numero nota fiscal'].unique()
    total_notas_problema = len(notas_problematicas)
    
    print(f"🔍 Encontradas {total_notas_problema} notas com itens onde preço de custo > valor unitário")
    print(f"📦 Itens problemáticos identificados: {len(itens_problematicos)}")
    
    # 3. Criar versão limpa (excluindo as notas problemáticas)
    df_limpo = df[~df['Numero nota fiscal'].isin(notas_problematicas)].copy()
    
    # 4. Criar dataframe com as notas removidas (para auditoria)
    df_removido = df[df['Numero nota fiscal'].isin(notas_problematicas)].copy()
    
    # 5. Recalcular valores totais na base limpa
    df_limpo['Valor total produto'] = df_limpo['Quantidade do produto'] * df_limpo['Valor unitário']
    df_limpo['Valor da nota'] = df_limpo.groupby('Numero nota fiscal')['Valor total produto'].transform('sum')
    
    # 6. Relatório de impacto
    print("\n✅ RESULTADO DA LIMPEZA:")
    print(f"- Notas originais: {df['Numero nota fiscal'].nunique()}")
    print(f"- Notas removidas: {total_notas_problema}")
    print(f"- Notas restantes: {df_limpo['Numero nota fiscal'].nunique()}")
    print(f"- Registros removidos: {len(df_removido)}")
    print(f"- Percentual removido: {len(df_removido)/len(df):.2%}")
    
    return df_limpo, df_removido

# Como usar:
# df_limpo, df_removido = excluir_notas_com_custo_maior_que_venda(df_mesclado)

# Para exportar os resultados:
# df_limpo.to_excel('base_limpa.xlsx', index=False)
# df_removido.to_excel('notas_removidas_auditoria.xlsx', index=False)

In [104]:
df_limpo, df_removido = excluir_notas_com_custo_maior_que_venda(df_completo_limpo)

🔍 Encontradas 275 notas com itens onde preço de custo > valor unitário
📦 Itens problemáticos identificados: 446

✅ RESULTADO DA LIMPEZA:
- Notas originais: 24285
- Notas removidas: 275
- Notas restantes: 24010
- Registros removidos: 3128
- Percentual removido: 1.77%


In [105]:
df_limpo.head(10)

,Numero nota fiscal,Data da venda,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Preço de custo,Valor total produto,Valor da nota,Código da categoria,Categoria,Código da Marca,Marca,Quantidade estoque
0,566082,2025-06-17,33932,ENV 229X324 OU C/100 FILIPERSON,2,33.00,24.41,66.00,4092.73,47,ENVELOPES ( ENV ),8389,FILIPERSON NACIONAL REFL IND COMÉRCIO DE PAP E...,2076
1,566082,2025-06-17,29367,GUARDANAPO SNOB FS 33X30 50F SANTHER,5,2.90,2.23,14.50,4092.73,27,GUARDANAPOS ( GDP ),8340,SANTHER FABRICA DE PAPEL SANTA THEREZINHA S/A,3475
2,566082,2025-06-17,241,PASTA AZ OF LE TIGRADA FRAMA 356-11952,40,9.45,7.61,378.00,4092.73,38,PASTAS ( PST ),185,FRAMA INDUSTRIA GRAFICA LTDA,2104
3,566082,2025-06-17,32608,CALCULADORA MESA 12 DIG MASTERPRINT 1086,6,20.87,17.05,125.22,4092.73,147,CALCULADORAS ( CLC ),4281,GRUPO MASTER COMERCIO IMPORTACAO E EXPORTACAO ...,61
4,566082,2025-06-17,25495,MARCA TEXTO AM C/12 MASTERPRINT MP612,2,11.30,8.72,22.60,4092.73,389,MARCA TEXTO ( MRT),4281,GRUPO MASTER COMERCIO IMPORTACAO E EXPORTACAO ...,293
5,566082,2025-06-17,23502,COLA EVA S/TOLUENO 75G BRASCOLA,1,5.20,3.94,5.20,4092.73,704,COLAS ( COL ),3612,BRASCOLA LTDA,460
6,566082,2025-06-17,23717,PERFURADOR 2F 35FL METAL BRW PF3000,2,42.00,32.40,84.00,4092.73,81,PERFURADORES ( PRF ),4490,BRW SUPRIMENTOS ESCOLARES E ESCRITORIO LTDA.,394
7,566082,2025-06-17,24749,COLA INSTANTANEA 5G SUPER COLA TEKBOND,5,6.26,4.65,31.30,4092.73,704,COLAS ( COL ),8690,SAINT-GOBAIN DO BRASIL PRODUTOS IND. E P/ CONS...,323
8,566082,2025-06-17,23306,ELASTICO AM N°18 C/100 MERCUR,1,4.82,3.72,4.82,4092.73,124,ELASTICOS ( ELA ),1012,MERCUR SA,1977
9,566082,2025-06-17,6681,ELASTICO FINO Nº18 AM C/2200 MERCUR,1,79.83,57.96,79.83,4092.73,124,ELASTICOS ( ELA ),1012,MERCUR SA,106


In [106]:
df_limpo.shape

(173999, 14)

In [107]:
import pandas as pd

def verificar_base_simples(df):
    """Verificação simplificada da base mesclada"""
    print("🔍 VERIFICAÇÃO SIMPLIFICADA DA BASE")
    
    # 1. Verificar valores problemáticos
    problemas = {
        'Valores nulos': df.isnull().sum().sum(),
        'Valores zerados': (df.select_dtypes(include='number') == 0).sum().sum(),
        'Valores negativos': (df.select_dtypes(include='number') < 0).sum().sum(),
        'Preço custo > venda': (df['Preço de custo'] > df['Valor unitário']).sum(),
        'Cálculos incorretos': (df['Valor total produto'] != (df['Quantidade do produto'] * df['Valor unitário'])).sum()
    }
    
    # 2. Mostrar resultados
    print("\n📊 PROBLEMAS ENCONTRADOS:")
    for tipo, qtd in problemas.items():
        if qtd > 0:
            print(f"- {tipo}: {qtd} ocorrências")
    
    # 3. Recomendação final
    if any(qtd > 0 for qtd in problemas.values()):
        print("\n⚠️ RECOMENDAÇÃO: A base precisa de tratamento adicional!")
    else:
        print("\n✅ A base está pronta para uso!")

# Como usar:
# verificar_base_simples(df_mesclado)

In [109]:
verificar_base_simples(df_limpo)

🔍 VERIFICAÇÃO SIMPLIFICADA DA BASE

📊 PROBLEMAS ENCONTRADOS:
- Valores zerados: 3544 ocorrências
- Valores negativos: 6 ocorrências

⚠️ RECOMENDAÇÃO: A base precisa de tratamento adicional!


In [110]:
def encontrar_valores_problematicos(df):
    """
    Encontra valores zerados e negativos nas colunas numéricas
    Retorna DataFrames separados com os registros problemáticos
    """
    # Seleciona apenas colunas numéricas
    colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    
    print("🔍 PROCURANDO VALORES ZERADOS E NEGATIVOS")
    
    # 1. Encontrar valores zerados
    zeros = {}
    registros_zerados = pd.DataFrame()
    
    for col in colunas_numericas:
        qtd_zeros = (df[col] == 0).sum()
        if qtd_zeros > 0:
            zeros[col] = qtd_zeros
            # Adiciona os registros zerados
            registros_zerados = pd.concat([registros_zerados, df[df[col] == 0]])
    
    print(f"\nVALORES ZERADOS ENCONTRADOS: {sum(zeros.values())}")
    for col, qtd in zeros.items():
        print(f"- {col}: {qtd}")
    
    # 2. Encontrar valores negativos
    negativos = {}
    registros_negativos = pd.DataFrame()
    
    for col in colunas_numericas:
        qtd_neg = (df[col] < 0).sum()
        if qtd_neg > 0:
            negativos[col] = qtd_neg
            # Adiciona os registros negativos
            registros_negativos = pd.concat([registros_negativos, df[df[col] < 0]])
    
    print(f"\nVALORES NEGATIVOS ENCONTRADOS: {sum(negativos.values())}")
    for col, qtd in negativos.items():
        print(f"- {col}: {qtd}")
    
    return registros_zerados, registros_negativos

# Como usar:
# zerados, negativos = encontrar_valores_problematicos(df_mesclado)

# Para ver os primeiros registros com valores zerados:
# print(zerados.head())

# Para ver os registros com valores negativos:
# print(negativos.head())

# Para exportar os resultados:
# zerados.to_excel('valores_zerados.xlsx', index=False)
# negativos.to_excel('valores_negativos.xlsx', index=False)

In [112]:
# Como usar:
zerados, negativos = encontrar_valores_problematicos(df_limpo)

# Para ver os primeiros registros com valores zerados:
print(zerados.head(2))

# Para ver os registros com valores negativos:
print(negativos.head(2))

🔍 PROCURANDO VALORES ZERADOS E NEGATIVOS

VALORES ZERADOS ENCONTRADOS: 3544
- Preço de custo: 1
- Valor da nota: 6
- Quantidade estoque: 3537

VALORES NEGATIVOS ENCONTRADOS: 6
- Quantidade do produto: 3
- Valor total produto: 3
        Numero nota fiscal Data da venda  Código produto  \
101175              550463    2025-02-25           19131   
177829                 496    2025-04-01            6620   

                           Descrição do produto  Quantidade do produto  \
101175                 PORTA RETRATO DPLAST 169                     10   
177829  BAND ISOPOR B3 BR C/400 COPOBRAS CRL003                     50   

        Valor unitário  Preço de custo  Valor total produto  Valor da nota  \
101175            2.19            0.00                 21.9         265.66   
177829            8.83            8.83                441.5           0.00   

        Código da categoria                  Categoria  Código da Marca  \
101175                  598      PORTA RETRATO ( PTA )    

In [113]:
import pandas as pd

def verificar_notas_afetadas(df):
    """
    Verifica quantas notas fiscais são afetadas por valores zerados e negativos
    """
    print("📊 NOTAS FISCAIS AFETADAS POR VALORES PROBLEMÁTICOS")
    
    # 1. Verificar valores zerados
    print("\n🔍 VALORES ZERADOS:")
    # Preço de custo zerado
    notas_custo_zerado = df[df['Preço de custo'] == 0]['Numero nota fiscal'].nunique()
    print(f"- Notas com Preço de custo zerado: {notas_custo_zerado}")
    
    # Valor da nota zerado
    notas_valor_zerado = df[df['Valor da nota'] == 0]['Numero nota fiscal'].nunique()
    print(f"- Notas com Valor da nota zerado: {notas_valor_zerado}")
    
    # 2. Verificar valores negativos
    print("\n🔍 VALORES NEGATIVOS:")
    # Quantidade do produto negativa
    notas_qtde_negativa = df[df['Quantidade do produto'] < 0]['Numero nota fiscal'].nunique()
    print(f"- Notas com Quantidade negativa: {notas_qtde_negativa}")
    
    # Valor total produto negativo
    notas_valor_negativo = df[df['Valor total produto'] < 0]['Numero nota fiscal'].nunique()
    print(f"- Notas com Valor total negativo: {notas_valor_negativo}")

# Como usar:
# verificar_notas_afetadas(df_mesclado)

In [114]:
verificar_notas_afetadas(df_limpo)

📊 NOTAS FISCAIS AFETADAS POR VALORES PROBLEMÁTICOS

🔍 VALORES ZERADOS:
- Notas com Preço de custo zerado: 1
- Notas com Valor da nota zerado: 1

🔍 VALORES NEGATIVOS:
- Notas com Quantidade negativa: 1
- Notas com Valor total negativo: 1


In [115]:
def excluir_notas_problematicas(df):
    """
    Exclui notas fiscais afetadas por:
    - Preço de custo zerado
    - Valor da nota zerado
    - Quantidade do produto negativa
    - Valor total produto negativo
    """
    # Identificar notas problemáticas
    notas_problema = set()
    
    # 1. Notas com Preço de custo zerado
    notas_problema.update(df[df['Preço de custo'] == 0]['Numero nota fiscal'].unique())
    
    # 2. Notas com Valor da nota zerado
    notas_problema.update(df[df['Valor da nota'] == 0]['Numero nota fiscal'].unique())
    
    # 3. Notas com Quantidade negativa
    notas_problema.update(df[df['Quantidade do produto'] < 0]['Numero nota fiscal'].unique())
    
    # 4. Notas com Valor total negativo
    notas_problema.update(df[df['Valor total produto'] < 0]['Numero nota fiscal'].unique())
    
    # Criar DataFrame limpo
    df_limpo = df[~df['Numero nota fiscal'].isin(notas_problema)].copy()
    
    # Relatório
    print(f"Notas originais: {df['Numero nota fiscal'].nunique()}")
    print(f"Notas removidas: {len(notas_problema)}")
    print(f"Notas restantes: {df_limpo['Numero nota fiscal'].nunique()}")
    print(f"Registros removidos: {len(df) - len(df_limpo)}")
    
    return df_limpo

# Como usar:
# df_limpo = excluir_notas_problematicas(df_mesclado)
# df_limpo.to_excel('base_limpa.xlsx', index=False)

In [116]:
df_limpo = excluir_notas_problematicas(df_limpo)

Notas originais: 24010
Notas removidas: 2
Notas restantes: 24008
Registros removidos: 16


In [118]:
df_limpo["Numero nota fiscal"].nunique()

24008

In [120]:
df_limpo.to_excel("bases/bases_limpas/base_para_modelo_final.xlsx", index=False)